In [1]:
import requests
from bs4 import BeautifulSoup
import time 
import random 
import re
from fake_useragent import UserAgent
import datetime
from elasticsearch import Elasticsearch
from tqdm import tqdm
es=Elasticsearch()

In [2]:
class KanobuArticle:
    def __init__(self):
        self.date=""
        self.time=""
        self.words=""
        self.rubr=""
        self.rubrid=""
        self.hashtag=""
        self.weblink=""
        self.author=""
        self.head=""
        self.text=""
        

    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "words":"'+self.words+'", "rubrics_id":"'+self.rubrid+'", "rubrics":"'+self.rubr+'", "hashtag":"'+self.hashtag+'", "title":"'
        +self.head+'", "link":"'+self.weblink+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res


    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics_id":self.rubrid, "rubrics":self.rubr, "hashtag":self.hashtag, \
             "title":self.head, "words":self.words, "link":self.weblink, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

In [3]:
def GetPagesArticles(number_of_pages):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    url = 'https://kanobu.ru/news/?page='
    links = []
    for page in range(1, number_of_pages+1):
        r = requests.get(url + str(page), headers = headers)
        time.sleep(random.randint(1,6))
        content = r.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
        
        for link in soup.find_all('a', attrs={'class':'c-item_foot d-b t-l'}):
            links.append(link.get('href'))
    
    return links    

In [4]:
def clean_text (text):
    text = text.replace('\xa0', ' ')
    text = text.replace ('\n', ' ')
    text = text.replace ('  ', ' ')
    return text

In [5]:
def add_article(links):
    l = 'https://kanobu.ru'
    articles = []
    rubrics_id = {}
    i = 1
    
    for link in links:
        art = KanobuArticle()
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
        page = requests.get(l+link, headers = headers)
        content = page.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
    
        art.weblink = l+link
    
        date = soup.find('div', attrs={'class':'c-title_footer'}).get_text()
        art.date = date.split(', ')[0]
        art.time = date.split(', ')[1]
        
        author = soup.find('a', attrs={'class':'c-title_author'}).get_text()
        art.author = author
        
        text = soup.find('div', attrs={'class':'c-detail_content'}).get_text()
        art.text = clean_text(text)
        
        words = len(clean_text(text).split())
        art.words = words
        
        title = soup.find('h1', attrs={'class':'c-title_body'}).get_text()
        art.head = clean_text(title)
        
        rubric = soup.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
        r = rubric.split(' | ')
        art.rubr = r[0]
    
        if r[0] not in rubrics_id:
            rubrics_id[r[0]] = i
            i+=1
        art.rubrid = rubrics_id[r[0]]
    
        try:
            art.hashtag = r[1]
        except:
            pass
    
        #print(art)
        #print(text)
        #print(date)
        #print(author)
        print(title)
        print(words)
        print(link)
        print(i)
        #print(r[0])
        #print(r[1])
    
    
        res = es.search(index="kanobu", body={"query": {"match_all": {}}})
        maxid = res['hits']['total']['value']
    
        es.index(index="kanobu", doc_type='article', id = maxid, body=art.toDict())

In [6]:
def create_db(index):
    es.indices.create(index=index)
    mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":" HH:mm"},
                                "words":{"type":"double"},
                                "hashtag":{"type":"text","analyzer" : "russian"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "rubricsid":{"type":"double"},
                                "link":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}
    es.indices.put_mapping(index=index, doc_type='article', body=mapit, include_type_name=True)

In [7]:
def GetArticles(links):
    create_db("kanobu")
    add_article(links)
    time.sleep(random.randint(1,3))

In [8]:
es.indices.delete(index="kanobu") 

{'acknowledged': True}

In [9]:
GetArticles(GetPagesArticles(2))

Джей Джей Абрамс и Epic Games сделали мобильную игру про шпионов. Вот первые скриншоты
207
/news/dzhej-dzhej-abrams-i-epic-games-sdelali-mobilnuyu-igru-pro-shpionov-vot-pervyie-skrinshotyi-422883/
2
Актеры GTA V призвали не верить инсайдам о GTA VI: «Откуда они берут это?»
169
/news/akteryi-gta-v-prizvali-ne-verit-insajdam-o-gta-vi-otkuda-oni-berut-eto-422882/
2
Гоблин правильно озвучил художественный фильм «Джентльмены» для «Ростелекома»
136
/news/goblin-pravilno-ozvuchil-hudozhestvennyij-film-dzhentlmenyi-dlya-rostelekoma-422881/
3
Звезда «Достать ножи» советует «Надеть маски». Джейми Ли Кертис рекомендует сидеть на карантине
130
/news/zvezda-dostat-nozhi-sovetuet-nadet-maski-dzhejmi-li-kertis-rekomenduet-sidet-na-karantine-422880/
3
ГеймНет-челлендж: вебинар с организаторами
218
/news/gejmnet-chellendzh-vebinar-sorganizatorami-422879/
3
Инсайдер рассказал про Resident Evil 8. Изначально это была Revelations 3
186
/news/insajder-rasskazal-pro-resident-evil8-iznachalno-eto-byila-revel

### Интерфейс

In [10]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

In [11]:
text1 = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [12]:
menu = widgets.Dropdown(
       options=['id', 'title', 'date', 'author', 'hashtag', 'rubrics'],
       value='id',
       description='How to find?')

In [13]:
text = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [14]:
text2 = widgets.Text(
    value='',
    placeholder='Enter the link of the article to add',
    description='Text:',
    disabled=False
)

In [15]:
order = widgets.Dropdown(
       options=['ascending', 'descending'],
       value='ascending',
       description='Order')

In [16]:
sortby = widgets.Dropdown(
       options=['_id', 'words'],
       value='_id',
       description='Sort by')

In [17]:
number = widgets.IntSlider(
    value=10,
    min=1,
    max=10000,
    step=1,
    description='How many articles?',
)

In [18]:
counter = widgets.Dropdown(
       options=['min', 'max', 'avg'],
       value='max',
       description='words')

In [19]:
button1 = widgets.Button(description='Count')
out = widgets.Output()

In [20]:
def on_button_clicked1(_):
    with out:
        clear_output()
        res=es.search(index = "kanobu", body = {"size": 0, "aggs": {counter.value: {counter.value: {"field": "words" }}}})
        print(round(res["aggregations"][counter.value]["value"]))
        

In [21]:
button1.on_click(on_button_clicked1)

In [22]:
button2 = widgets.Button(description='Match with options')
out = widgets.Output()

def on_button_clicked2(_):
    with out:
        clear_output()
        if menu.value == 'id':
            id_search = int(text.value)
            print(es.get(index="kanobu", doc_type='article', id=id_search))
        
        else:
            res = es.search(index="kanobu", body={"query": {"match": {menu.value : text.value}}}, size=100)
            for part in res["hits"]["hits"]:
                print(part)

In [23]:
button2.on_click(on_button_clicked2)

In [24]:
button3 = widgets.Button(description='Add article')
out = widgets.Output()

def on_button_clicked3(_):
    with out:
        clear_output()
        link = text2.value
        add_article(link)

In [25]:
button3.on_click(on_button_clicked3)

In [26]:
text3 = widgets.Text(
    value='',
    placeholder='Enter article id to delete',
    description='Text:',
    disabled=False
)

In [27]:
button4 = widgets.Button(description='Delete article')
out = widgets.Output()

def on_button_clicked4(_):
    with out:
        clear_output()
        id_del = int(text3.value)
        es.delete(index="kanobu",doc_type="article",id=id_del)
        print('Done')

In [28]:
button4.on_click(on_button_clicked4)

In [29]:
button5 = widgets.Button(description='Sort articles')
out = widgets.Output()

In [30]:
def on_button_clicked5(_):
    with out:
        clear_output()
        if order.value == 'ascending':
            ord = str(sortby.value)+':asc'
        elif order.value == 'descending':
            ord = str(sortby.value)+':desc'
        res = es.search(index="kanobu", body={"query": {"match_all": {}}}, sort=ord,size=number.value)
        for part in res["hits"]["hits"]:
            print(part)

In [31]:
button5.on_click(on_button_clicked5)

In [32]:
id_upd = widgets.Text(
    value='',
    placeholder='Enter your id',
    description='Text:',
    disabled=False
)

menu_upd = widgets.Dropdown(
       options=['title', 'date', 'author', 'hashtag', 'rubrics', 'text'],
       value='title',
       description='Update')

text_upd = widgets.Text(
    value='',
    placeholder='Enter your update',
    description='Text:',
    disabled=False
)

button_upd = widgets.Button(description='Update')
out = widgets.Output()

def on_button_clicked_upd(_):
    with out:
        clear_output()
        your_id = int(id_upd.value)
        upd_area = str(menu_upd.value)
        text = str(text_upd.value)
        es.update(index='kanobu', doc_type='article', id=your_id, body={"doc": {upd_area: text}})
        print('Done')

In [33]:
button_upd.on_click(on_button_clicked_upd)

In [45]:
number_size = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='How many articles to show?',
)

In [46]:
number_in_group = widgets.IntSlider(
    value=10,
    min=1,
    max=30,
    step=1,
    description='How many articles in a group?',
)

In [47]:
button_group = widgets.Button(description='Group by rubrics')
out = widgets.Output()

In [48]:
def on_button_clicked_group(_):
    with out:
        clear_output()
        res = es.search(index="kanobu", body={"size":number_size.value, "query": {"match_all": {}}, "collapse": {"field" : "rubrics_id", "inner_hits": {"name":"rubrics_id","size": number_in_group.value}}})
        for group in res['hits']['hits']:
            print(group)
            print()
        

In [49]:
button_group.on_click(on_button_clicked_group)

In [50]:
box1 = widgets.VBox([menu, text, button2])
box2 = widgets.VBox([text2, button3, text3, button4])
box3 = widgets.VBox([id_upd, menu_upd, text_upd, button_upd])
box4 = widgets.VBox([sortby,order, number, button5])
box5 = widgets.VBox([counter, button1])
box6 = widgets.VBox([number_size, number_in_group, button_group])
widgets.VBox([box1, box2, box3, box4, box5, box6, out])

- поиск по Id выполняется по другому (придумать как добавить его в ту же кнопку) - <b>сделано</b>
- кнопка для добавления статьи по ссылке - <b>сделано</b>
- не хватает операции update (например, "положи по существующему индексу другую статью" - <b>сделано</b>
- нужно организовать меню с кнопками получше - <b>сделано</b>
- сортировка <b>сделано</b>
- - "2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...) " <b>сделано</b>
- виджет с агрегацией <b>сделано</b>
- Когда ищу для 2 апреля сначала идут статьи 2 апреля, а потом почему-то статьи 3 апреля (и наоборот)
- группировка <b>сделано<b>

"Требования к нереляционным БД
2 - нереляционная БД
2 - красивая структура БД
2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)
2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)
2 - зависит от БД: Redis - использование ключей, хешей, ...; Neo4j - нахождение путей не только к соседним вершинам, операции с графами; MongoDB, ElasticSearch - работа с текстами или географией; остальные БД - будем договариваться."